In [258]:
import numpy as np
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import os
import folium
from folium.plugins import MarkerCluster



In [259]:
maps = os.listdir("ml/data/map")
maps.remove("us_map.npy")
maps_dict = {map.replace(".npy",""): np.load(f"ml/data/map/{map}") for map in maps}
xmap = np.load("ml/data/map/us_map.npy")[::-1]

In [260]:
clean = os.listdir("data/clean")
clean_dict = {file.replace(".csv",""): pd.read_csv(f"data/clean/{file}") for file in clean}
clean = list(clean_dict.keys())

In [267]:
cities = pd.read_csv("american_cities.csv").drop(columns=["Unnamed: 0"])
cities = cities.drop_duplicates(keep="first", subset=["city"])
cities.set_index("city", inplace=True)

In [272]:
CITY = {"name": "Kansas City"}
CITY["lat"] = cities.loc[CITY["name"]]["lat"]
CITY["lon"] = cities.loc[CITY["name"]]["lng"]

colors = ["#f8b195", "#f67280", "#C06C84", "#6C5B7B", "#355C7D"]

In [275]:
if CITY["name"] == "Kansas City":
    CITY["components"] = [clean_dict["cng_stations_clean"].sample(1000), clean_dict["docks_clean"].sample(500)]

In [296]:
import branca

legend_html = '''
{% macro input(CITY) %}
<div style="
    position: fixed; 
    bottom: 50px;
    left: 50px;
    width: 500px;
    height: 80px;
    z-index:9999;
    font-size:14px;
    ">
    {% for i in enumerate(CITY["components"])%}
        <p><a style="color:#f8b195;font-size:150%;margin-left:20px;">&#9864;</a>&emsp;{}</p>
    {% endfor %}
    <p><a style="color:#f67280;font-size:150%;margin-left:20px;">&#9864;</a>&emsp;{CITY["components"].keys[1]}</p>
</div>
<div style="
    position: fixed; 
    bottom: 50px;
    left: 50px;
    width: 250px;
    height: 80px; 
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;

    opacity: 0.7;
    ">
</div>
{% endmacro %}
'''
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)

map = folium.Map(location=[CITY["lat"], CITY["lon"]], zoom_start=8,max_bounds=True, min_zoom=8, tiles="cartodbpositron")

lat_bounds = 4
lon_bounds = 4

cities = cities.loc[(cities["lat"]>CITY["lat"]-lat_bounds) & (cities["lat"]<CITY["lat"]+lat_bounds) & (cities["lng"]>CITY["lon"]-lon_bounds) & (cities["lng"]<CITY["lon"]+lon_bounds)][:10]
for city in cities.index:
    folium.Circle([cities.loc[city]["lat"], cities.loc[city]["lng"]], radius=cities.loc[city]["population"]/300, popup=f"{city} population: {cities.loc[city]['population']}", color="black", fill=True,
 fill_color="black", opacity=0.2).add_to(map)
 
#diplay points on map
for ind, component in enumerate(CITY["components"]):
    
    for index, row in component.loc[(component["latitude"]>CITY["lat"]-lat_bounds) & (component["latitude"]<CITY["lat"]+lat_bounds) & (component["longitude"]>CITY["lon"]-lon_bounds) & (component["longitude"]<CITY["lon"]+lon_bounds)].iterrows():
        folium.CircleMarker([row["latitude"], row["longitude"]], radius=2, color=colors[ind], fill=True, fill_opacity=1).add_to(map)

folium.LayerControl().add_to(map)
map.get_root().add_child(legend)
map

In [214]:
for val in cities.iterrows():
    print(type(val))
    break

<class 'tuple'>


In [215]:
cities.loc["Portland"]

lat                 45.5372
lng                 -122.65
country       United States
population          2052796
Name: Portland, dtype: object

In [216]:
clean_dict["elec_stations_clean"].shape

(56740, 3)

In [238]:
clean_dict["elec_stations_clean"]

,longitude,latitude,val
0,-118.387971,34.248319,1
1,-118.448504,34.052542,1
2,-118.271387,34.040539,1
3,-118.248589,34.059133,1
4,-118.096665,33.759802,1
...,...,...,...
56735,-88.759611,44.622129,1
56736,-88.759701,44.622163,1
56737,-84.477325,34.677950,1
56738,-70.895300,42.499916,1
